In [1]:
print("testing")

testing


In [2]:
%pwd

'/Users/georgelin/Desktop/Projects/medical_chat_LLM/research'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [7]:
# extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(data, 
                             glob = "*.pdf", 
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [8]:
extracted_data = load_pdf_files("data")

FileNotFoundError: Directory not found: 'data'